## Mapa

In [1]:
import pandas as pd
import folium

# Ruta al archivo JSON
file_path = "dataset/transacciones_actualizadas.json"

# Cargar el JSON como lista
try:
    df = pd.read_json(file_path)  # No usar `lines=True`
    print("Datos cargados exitosamente.")
except ValueError as e:
    print(f"Error al cargar el archivo JSON: {e}")
    raise

# Verificar si el dataset tiene datos
if df.empty:
    raise ValueError("El archivo JSON no contiene datos o está vacío.")

# Crear un mapa centrado en las coordenadas promedio del dataset
centro_latitud = df["latitud"].mean()
centro_longitud = df["longitud"].mean()
mapa = folium.Map(location=[centro_latitud, centro_longitud], zoom_start=12)

# Añadir marcadores para cada transacción en el dataset
for _, fila in df.iterrows():
    popup_info = f"<b>ID Transacción:</b> {fila['id_transaccion']}<br>" \
                 f"<b>Monto:</b> {fila['monto']}<br>" \
                 f"<b>Ordenante:</b> {fila['ordenante_nombre_completo']}<br>" \
                 f"<b>Beneficiario:</b> {fila['beneficiario_nombre_completo']}"

    folium.Marker(
        location=[fila["latitud"], fila["longitud"]],
        popup=folium.Popup(popup_info, max_width=300),
        icon=folium.Icon(color="blue", icon="info-sign")
    ).add_to(mapa)

# Guardar el mapa en un archivo HTML
output_path = "mapa_transacciones.html"
mapa.save(output_path)
print(f"Mapa interactivo guardado en: {output_path}")


Datos cargados exitosamente.
Mapa interactivo guardado en: mapa_transacciones.html


## Preprocesamiento y normalizacion del dataset

In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Cargar el dataset desde un archivo JSON
file_path = r"dataset/transacciones_actualizadas.json"
df = pd.read_json(file_path)

# Filtrar las columnas relevantes
variables_interes = ["ordenante_cedula", "latitud", "longitud", "fecha_hora", "local_nombre"]
df = df[variables_interes]

# Completar valores faltantes en 'local_nombre'
df["local_nombre"] = df["local_nombre"].fillna("Desconocido")

# Guardar las columnas originales de latitud y longitud
df["latitud_original"] = df["latitud"]
df["longitud_original"] = df["longitud"]

# Extraer hora y día de la semana de 'fecha_hora'
df["hora"] = pd.to_datetime(df["fecha_hora"]).dt.hour
df["dia_semana"] = pd.to_datetime(df["fecha_hora"]).dt.weekday

# Eliminar la columna original 'fecha_hora' ya que ahora está representada
df = df.drop(columns=["fecha_hora"])

# Codificar 'local_nombre' con LabelEncoder para convertirlo en una variable numérica
label_encoder = LabelEncoder()
df["local_nombre_encoded"] = label_encoder.fit_transform(df["local_nombre"])

# Mostrar el dataset transformado
print("Dataset transformado con columnas originales de latitud y longitud:")
print(df.head())

# Guardar el dataset transformado
output_path = "dataset/transacciones_transformadas.json"
df.to_json(output_path, orient="records", lines=True)
print(f"Dataset transformado guardado en: {output_path}")

Dataset transformado con columnas originales de latitud y longitud:
   ordenante_cedula    latitud   longitud      local_nombre  latitud_original  \
0        3597400581 -12.046357 -77.042883       Desconocido        -12.046357   
1        3597400581 -12.046464 -77.042727       Desconocido        -12.046464   
2        3597400581 -12.046440 -77.042898       Desconocido        -12.046440   
3        3597400581  -3.995926 -79.204172  Bogati Heladeria         -3.995926   
4        3597400581  -4.007181 -79.203194       Magic Retro         -4.007181   

   longitud_original  hora  dia_semana  local_nombre_encoded  
0         -77.042883     2           6                     3  
1         -77.042727     0           2                     3  
2         -77.042898    15           3                     3  
3         -79.204172     7           3                     0  
4         -79.203194    18           3                     7  
Dataset transformado guardado en: dataset/transacciones_transformad

## Dividir el dataset para entrenamiento y para testeo

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Cargar el dataset transformado
file_path = "dataset/transacciones_transformadas.json"
df = pd.read_json(file_path, lines=True)

# Dividir las transacciones de cada usuario
df_train = pd.DataFrame()
df_test = pd.DataFrame()

for cedula, group in df.groupby("ordenante_cedula"):
    # Dividir las transacciones del usuario en entrenamiento y testeo (80-20)
    group_train, group_test = train_test_split(group, test_size=0.2, random_state=42)
    df_train = pd.concat([df_train, group_train])
    df_test = pd.concat([df_test, group_test])

# Guardar los datasets en archivos separados
train_path = "dataset/transacciones_train.json"
test_path = "dataset/transacciones_test.json"

df_train.to_json(train_path, orient="records", lines=True)
df_test.to_json(test_path, orient="records", lines=True)

# Imprimir mensajes de confirmación
print(f"Dataset de entrenamiento guardado en: {train_path}")
print(f"Dataset de testeo guardado en: {test_path}")

# Verificar ejemplos de los conjuntos
print("Ejemplo del dataset de entrenamiento:")
print(df_train.head())
print("\nEjemplo del dataset de testeo:")
print(df_test.head())


Dataset de entrenamiento guardado en: dataset/transacciones_train.json
Dataset de testeo guardado en: dataset/transacciones_test.json
Ejemplo del dataset de entrenamiento:
     ordenante_cedula    latitud   longitud            local_nombre  \
462        2250834721 -12.046400 -77.042874             Desconocido   
454        2250834721  -4.002812 -79.203544  Cafeteria Repe y Bolon   
487        2250834721  -3.995926 -79.204172        Bogati Heladeria   
458        2250834721  -4.001608 -79.208955     La Nona Restraurant   
453        2250834721  -4.001608 -79.208955     La Nona Restraurant   

     latitud_original  longitud_original  hora  dia_semana  \
462        -12.046400         -77.042874    19           5   
454         -4.002812         -79.203544    13           4   
487         -3.995926         -79.204172     2           3   
458         -4.001608         -79.208955    19           0   
453         -4.001608         -79.208955    14           6   

     local_nombre_encoded  


## Creacion de Modelo KNN

Se implementa el entrenamiento del modelo K-Nearest Neighbors (KNN) utilizando el historial de transacciones de los usuarios. El modelo KNN permite encontrar los vecinos más cercanos basándose en las características relevantes de las transacciones.

In [4]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
import joblib

# Cargar el dataset transformado y normalizado
train_path = "dataset/transacciones_train.json"
df_train = pd.read_json(train_path, lines=True)

# Definir los pesos para las características
pesos = {
    "latitud": 0.3,
    "longitud": 0.3,
    "hora": 0.3,
    "dia_semana": 0.1
}

# Aplicar los pesos a las características normalizadas
for columna, peso in pesos.items():
    df_train[columna] *= peso

# Seleccionar las características para entrenar el modelo
X_train = df_train[["latitud", "longitud", "hora", "dia_semana"]]

# Entrenar el modelo KNN
modelo_knn = NearestNeighbors(n_neighbors=10, metric="euclidean")
modelo_knn.fit(X_train)

NearestNeighbors(metric='euclidean', n_neighbors=10)

### Se guarda el modelo


In [5]:
# Guardar el modelo entrenado
joblib.dump(modelo_knn, "recomendador_knn.pkl")

print("Modelo KNN entrenado y guardado correctamente.")


Modelo KNN entrenado y guardado correctamente.


## Función para Recomendar

Se implementa una función para realizar recomendaciones personalizadas basadas en el historial de transacciones de un usuario. La función combina el uso del algoritmo K-Nearest Neighbors (KNN) con un filtrado por hora (considerando una tolerancia especificada) y un cálculo de distancia Haversine para refinar las recomendaciones.

In [6]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic
import joblib

# Pesos para las características (aplicados antes de usar KNN)
pesos = {
    "latitud": 0.3,
    "longitud": 0.3,
    "hora": 0.3,
    "dia_semana": 0.1
}

# Cargar el modelo KNN previamente entrenado
modelo_knn = joblib.load("recomendador_knn.pkl")

def recomendar_por_cedula(cedula, cliente, radio_haversine=10.0, n_neighbors=5, tolerancia_horas=2):
    """
    Realiza recomendaciones basadas en el historial de transacciones del usuario.
    :param cedula: Cédula del usuario.
    :param cliente: Lista con las características del cliente (latitud, longitud, hora, día de la semana).
    :param radio_haversine: Radio en kilómetros para filtrar recomendaciones.
    :param n_neighbors: Número de vecinos a considerar con KNN.
    :param tolerancia_horas: Tolerancia en horas para filtrar por tiempo.
    :return: Recomendaciones de comercios dentro del radio.
    """
    try:
        # Cargar el dataset de entrenamiento
        train_path = "dataset/transacciones_train.json"
        df_train = pd.read_json(train_path, lines=True)

        # Filtrar el historial del usuario por cédula
        historial_usuario = df_train[df_train["ordenante_cedula"] == cedula]
        if historial_usuario.empty:
            print(f"No se encontró historial para el usuario con cédula: {cedula}")
            return None

        # Filtrar por hora con tolerancia
        hora_cliente = cliente[2]  # Hora del cliente
        historial_filtrado = historial_usuario[abs(historial_usuario["hora"] - hora_cliente) <= tolerancia_horas]

        if historial_filtrado.empty:
            print(f"No se encontraron transacciones cercanas en el tiempo para el usuario con cédula: {cedula}")
            return None

        # Aplicar los pesos a los datos del cliente
        cliente_datos = [
            cliente[0] * pesos["latitud"],
            cliente[1] * pesos["longitud"],
            cliente[2] * pesos["hora"],
            cliente[3] * pesos["dia_semana"]
        ]

        # Obtener los vecinos más cercanos utilizando KNN
        X_train = historial_filtrado[["latitud", "longitud", "hora", "dia_semana"]].copy()
        for columna, peso in pesos.items():
            X_train[columna] *= peso

        modelo_knn.fit(X_train)  # Entrenar el modelo con el historial filtrado del usuario
        distancias, indices = modelo_knn.kneighbors([cliente_datos], n_neighbors=n_neighbors)

        # Recuperar las recomendaciones iniciales
        recomendaciones = historial_filtrado.iloc[indices[0]].copy()

        # Calcular la distancia Haversine utilizando las columnas originales
        cliente_coords = (cliente[0], cliente[1])  # Coordenadas originales del cliente
        recomendaciones["distancia_haversine"] = recomendaciones.apply(
            lambda row: geodesic(
                (row["latitud_original"], row["longitud_original"]),
                cliente_coords
            ).kilometers,
            axis=1
        )

        # Filtrar las recomendaciones dentro del radio Haversine
        recomendaciones_filtradas = recomendaciones[recomendaciones["distancia_haversine"] <= radio_haversine]

        if recomendaciones_filtradas.empty:
            print(f"No se encontraron recomendaciones dentro del radio de {radio_haversine} km.")
            return None

        # Retornar las recomendaciones finales
        return recomendaciones_filtradas[[
            "local_nombre", "latitud_original", "longitud_original", 
            "hora", "dia_semana", "distancia_haversine"
        ]]

    except Exception as e:
        print(f"Error en la función de recomendación: {str(e)}")
        return None

# Datos del cliente: características originales
cedula_cliente = 4557997274
cliente_datos = [-3.995926, -79.204172, 10, 1]  # Coordenadas originales y contexto del cliente

# Radio Haversine en kilómetros
radio_haversine = 0.1

# Realizar la recomendación
recomendaciones = recomendar_por_cedula(cedula_cliente, cliente_datos, radio_haversine=radio_haversine)

# Imprimir las recomendaciones
if recomendaciones is not None:
    print("Recomendaciones finales:")
    print(recomendaciones)


c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Recomendaciones finales:
         local_nombre  latitud_original  longitud_original  hora  dia_semana  \
295  Bogati Heladeria         -3.995926         -79.204172    10           1   

     distancia_haversine  
295             0.000032  


## Fase de Testeo y Precisión


Iteración por cada transacción en el conjunto de testeo: Para cada transacción:

El modelo genera una lista de recomendaciones (los vecinos más cercanos según KNN).
Se verifica si el local_nombre real de la transacción aparece en las recomendaciones.
Suma de aciertos: Si el local_nombre está en las recomendaciones, se cuenta como un acierto.

Cálculo final:

Se divide el número de aciertos entre el total de transacciones en el conjunto de testeo.
El resultado se multiplica por 100 para obtener el porcentaje.

In [7]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from geopy.distance import geodesic

# Cargar el dataset de entrenamiento y testeo
train_path = "dataset/transacciones_train.json"
test_path = "dataset/transacciones_test.json"
df_train = pd.read_json(train_path, lines=True)
df_test = pd.read_json(test_path, lines=True)

# Pesos para las características
pesos = {
    "latitud": 0.3,
    "longitud": 0.3,
    "hora": 0.2,
    "dia_semana": 0.2
}

# Entrenar el modelo con el conjunto de entrenamiento
X_train = df_train[["latitud", "longitud", "hora", "dia_semana"]].copy()
for columna, peso in pesos.items():
    X_train[columna] *= peso
modelo_knn = NearestNeighbors(n_neighbors=5, metric="euclidean")
modelo_knn.fit(X_train)

# Calcular la precisión
aciertos = 0
total = 0

for cedula, group in df_test.groupby("ordenante_cedula"):
    # Filtrar historial del usuario en entrenamiento
    historial_usuario = df_train[df_train["ordenante_cedula"] == cedula]
    if historial_usuario.empty:
        continue  # Saltar usuarios sin historial en el conjunto de entrenamiento

    # Ajustar los datos del usuario en el conjunto de testeo
    for _, transaccion in group.iterrows():
        cliente_datos = [
            transaccion["latitud"] * pesos["latitud"],
            transaccion["longitud"] * pesos["longitud"],
            transaccion["hora"] * pesos["hora"],
            transaccion["dia_semana"] * pesos["dia_semana"]
        ]

        # Obtener los vecinos más cercanos utilizando KNN
        X_historial = historial_usuario[["latitud", "longitud", "hora", "dia_semana"]].copy()
        for columna, peso in pesos.items():
            X_historial[columna] *= peso
        modelo_knn.fit(X_historial)
        distancias, indices = modelo_knn.kneighbors([cliente_datos])

        # Comparar la recomendación más cercana con la transacción real
        recomendaciones = historial_usuario.iloc[indices[0]]
        if transaccion["local_nombre"] in recomendaciones["local_nombre"].values:
            aciertos += 1

        total += 1

# Calcular y mostrar la precisión
if total > 0:
    precision = (aciertos / total) * 100
    print(f"Precisión del modelo: {precision:.2f}% ({aciertos} aciertos de {total} transacciones)")
else:
    print("No hay datos suficientes para calcular la precisión.")


c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature n

Precisión del modelo: 75.00% (150 aciertos de 200 transacciones)


c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
c:\Practicum2.1\.venv\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
